In [59]:
import pandas as pd
import requests
from datetime import datetime
import time

In [65]:
owner = 'NixOS'  
repo = 'nixpkgs' 

url = f'https://api.github.com/repos/{owner}/{repo}/commits'    #endpoint commits
response = requests.get(url)
response.status_code

200

In [67]:
# first try with 1000 limit 

TOKEN = ''   #GitHub API Token

# Repository:
owner = 'NixOS'
repo = 'nixpkgs'
url = f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=100&page=21'   # Limit at 100 commits per request, ab seite 21

headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

# initialze dataframe 
commit_data = pd.DataFrame(columns=['Email', 'Timestamp', 'Company'])  

commit_limit = 3000  # limit 
commit_count = 0

# commit function
def get_commits(url, commit_count, commit_data):
    while url and commit_count < commit_limit:
        response = requests.get(url, headers=headers)
        
        try:
            response.raise_for_status()
            commits = response.json()

            for commit in commits:
                if commit_count >= commit_limit:
                    break
                author_url = commit['author']['url'] if commit['author'] else None
                email = commit['commit']['author']['email']
                timestamp = commit['commit']['author']['date']
                company = None

                # if author_url: vorhanden ist, try and get profilinfo for comopanyname
                if author_url:
                    user_response = requests.get(author_url, headers=headers)
                    if user_response.status_code == 200:
                        user_data = user_response.json()
                        company = user_data.get('company')  

                # append in df
                commit_data.loc[commit_count] = [email, timestamp, company]
                commit_count += 1

            # check next page
            url = response.links.get('next', {}).get('url')

            # waiting time between requests
            time.sleep(1)

        except requests.exceptions.HTTPError as err:
            print(f'HTTP Error: {err}')
            break
    return commit_count, commit_data

# Aufrufen der Funktion und Übergabe der Parameter
final_commit_count, final_commit_data = get_commits(url, 0, commit_data)

# Ausgabe der Ergebnisse
print(f'Total collected commits: {final_commit_count}')
final_commit_data
commit_data

Total collected commits: 1000


,Email,Timestamp,Company
0,gaetan@glepage.com,2024-11-02T22:59:15Z,Inria
1,mweinelt@users.noreply.github.com,2024-11-02T22:42:26Z,None
2,9713184+wegank@users.noreply.github.com,2024-11-02T22:29:01Z,@lip6
3,pyrox@pyrox.dev,2024-11-02T22:24:44Z,PyroNet
4,9713184+wegank@users.noreply.github.com,2024-11-02T22:19:08Z,@lip6
...,...,...,...
995,9713184+wegank@users.noreply.github.com,2024-11-01T02:55:40Z,@lip6
996,me@aleksana.moe,2024-11-01T02:55:27Z,UESTC
997,ryantm-bot@ryantm.com,2024-11-01T02:54:06Z,None
998,9713184+wegank@users.noreply.github.com,2024-11-01T02:53:37Z,@lip6


In [75]:
df2= commit_data

,Email,Timestamp,Company
0,gaetan@glepage.com,2024-11-02T22:59:15Z,Inria
1,mweinelt@users.noreply.github.com,2024-11-02T22:42:26Z,None
2,9713184+wegank@users.noreply.github.com,2024-11-02T22:29:01Z,@lip6
3,pyrox@pyrox.dev,2024-11-02T22:24:44Z,PyroNet
4,9713184+wegank@users.noreply.github.com,2024-11-02T22:19:08Z,@lip6
...,...,...,...
995,9713184+wegank@users.noreply.github.com,2024-11-01T02:55:40Z,@lip6
996,me@aleksana.moe,2024-11-01T02:55:27Z,UESTC
997,ryantm-bot@ryantm.com,2024-11-01T02:54:06Z,None
998,9713184+wegank@users.noreply.github.com,2024-11-01T02:53:37Z,@lip6


In [43]:
commit_data['Company'] = commit_data['Company'].str.replace('@', '', regex=False) #performs simple, direct string replacement(default r=True)
commit_data['Company'] = commit_data['Company'].str.strip()
commit_data['Company'].unique()

array(['NTNU', None, 'sportradar', 'UESTC', 'Inria', 'mathrills', 'sapcc',
       'Microsoft Norway', 'datakami', 'Infrastructure DBCDK', 'menzelit',
       'edgelesssys', 'Sammilani Mahavidyalaya', 'channable', 'numtide',
       'lip6', 'PyroNet', 'bitrise-io', 'SkylineTechnologies', 'Qumulo',
       'no', 'ObsidianSystems', 'Intel', 'eiicon-company',
       'mlabs-haskell', 'European Commission', 'Freelancer', 'reMarkable',
       'Helsinki Systems GmbH', 'Wavelens', 'hercules-ci',
       'Edgeless Systems', 'cz.nic labs CZ-NIC', 'secunet',
       'RawkodeAcademy', 'NixOS', 'UC Berkeley', 'Emotion Labs',
       'NextQuestion LLC', 'Epic Games / Mediatonic', 'fortx', 'sureapp',
       'wirenboard', 'Vrije Universiteit Brussel', 'dd-ix, tlm-solutions',
       'sensible.io', 'voltrondata', 'ClickHouse', 'deepin community',
       'Student caltech', 'hetzneronline', 'flyingcircusio'], dtype=object)

In [47]:
df1= commit_data

In [53]:
df1

,Email,Timestamp,Company
0,pbsds@hotmail.com,2024-11-05T19:45:17Z,NTNU
1,pbsds@hotmail.com,2024-11-05T19:37:59Z,NTNU
2,pbsds@hotmail.com,2024-11-05T19:32:27Z,NTNU
3,pbsds@hotmail.com,2024-11-05T19:23:10Z,NTNU
4,me@0upti.me,2024-11-05T18:23:38Z,None
...,...,...,...
995,tomoya.otabi@gmail.com,2024-11-02T14:02:20Z,None
996,9713184+wegank@users.noreply.github.com,2024-11-02T14:01:42Z,lip6
997,9713184+wegank@users.noreply.github.com,2024-11-02T14:01:21Z,lip6
998,9713184+wegank@users.noreply.github.com,2024-11-02T14:00:53Z,lip6


In [55]:
#df1.to_csv('cd1.csv')    #commit data 1 (first 1000)

In [79]:
#df2.to_csv('cd2.csv') # commit data 2 (next 1000) 

In [81]:
url = "https://api.github.com/rate_limit"
headers = {'Authorization': f'token {TOKEN}'}

response = requests.get(url, headers=headers)
rate_limit_info = response.json()

# Verbleibende Anfragen und Zeit bis zum Reset
remaining_requests = rate_limit_info['rate']['remaining']
reset_time = rate_limit_info['rate']['reset']
reset_time_local = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(reset_time))

print(f"Remaining requests: {remaining_requests}")
print(f"Limit reset time (local): {reset_time_local}")

Remaining requests: 3976
Limit reset time (local): 2024-11-06 08:12:58


In [ ]:
df24.duplicated().sum()

In [ ]:
df24= df24.drop_duplicates()

In [ ]:
df24.describe()

In [ ]:
df24['Company'].unique()

In [ ]:
df24['Company'] = df24['Company'].str.replace('@', '', regex=False) #performs simple, direct string replacement(default r=True)
df24['Company'] = df24['Company'].str.strip()

In [ ]:
df24['Company'].unique()